**Roll**: 19EC39034 \
**Name**: Shaswata Dutta \
**Project Code**: **PLHC-AS** \
**Project Title**: Premier League Player Rating using Single Linkage Agglomerative (Bottom-Up) Clustering Technique

Importing **necessary libraries**

In [1]:
# %%timeit
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import timeit
import time

### **Data Preprocessing**

In [3]:
# %%timeit
start_time = timeit.default_timer()
data_ = pd.read_csv('ipl.csv').to_numpy()
# print(data_.shape[0])
lab = data_[:, 0].reshape((data_.shape[0], 1))
# print(lab.shape)
data = data_[:, 1:]
# print(data.shape)
# print(data)
# print("Data type:", type(data))
elapsed = timeit.default_timer() - start_time
r, c = data.shape

## changing all data types to float except those of missing values
for i in range(r):
  for j in range(c):
    if(data[i][j] == '-'):
      continue
    else:
      data[i][j] = float(data[i][j])

## removing those columns having all entries as zeros
listcol = []
for i in range(c):
  cnt = 0
  for j in range(r):
    if(data[j, i] == 0):
      cnt =cnt+1
  if(cnt == r):
    listcol.append(i)
#     print(i)
data = np.delete(data, listcol, 1)
c = c-len(listcol)

print("Time elapsed for basic preprocessing: ", elapsed, "sec")

Time elapsed for basic preprocessing:  0.03984754200007501 sec


**DATA IMPUTATION** \
We have used three types of imputations to deal with the missing entries (the ones marked '-', which are of <class 'str'> type): 
 * Regression Imputation
 * Mean Imputation
 * Zero Imputation


**The required type of imputation to be used can be applied by running any of the following 3 code cells**.

**For the report, we have only used Regression Imputation.**

The following cell implements **Regression Imputation** for the empty values in the dataset.

In [4]:
start_time = timeit.default_timer()
def lin_reg(x_, y_):    # function for linear regression weight computation
  # we will add an extra 1 for including bias in the weights returned
  n = len(x_)
  x = np.zeros((n,2))
  for i in range(n):
    x[i][0] = 1        # adding an extra 1 for incorporating the bias term with the weights
    x[i][1] = x_[i]
  y = np.array(y_).reshape((n, 1))
  amat = np.matmul(np.transpose(x), x)
  bmat = np.matmul(np.transpose(x), y)
  w = np.matmul(np.linalg.inv(amat), bmat)
  return w

for j in range(c):
  flag = 0
  for i in range(r):
    if(data[i][j]=='-'):
      flag = 1
      break
  if(flag):
    # we will apply regression imputation on all the entries of column j
    print(j, flag)
    xarr = []
    yarr = []
    for i in range(r):
      if(isinstance(data[i][j], str)):
        continue
      else:
        xarr.append(data[i][j])
        yarr.append(i)
    warr = lin_reg(xarr, yarr)    # warr is a numpy array
    for i in range(r):
      if(isinstance(data[i][j], str)):
        data[i][j] = warr[0] + warr[1]*i

elapsed = timeit.default_timer() - start_time
print("Time elapsed for regression imputation on the entire dataset: ", elapsed, "sec")

5 1
17 1
19 1
Time elapsed for regression imputation on the entire dataset:  0.014216356000019914 sec


The following cell implements **Mean Imputation** for the empty values in the dataset.

In [ ]:
start_time = timeit.default_timer()
def mean_imput(x_, y_):   # for mean imputation
  n = len(x_)
  s = 0
  for i in x_:
    s = s+i
  return s/n

for j in range(c):
  flag = 0
  for i in range(r):
    if(data[i][j]=='-'):
      flag = 1
      break
  if(flag):
    # we will apply mean imputation on all the entries of column j
    print(j, flag)
    xarr = []
    yarr = []
    for i in range(r):
      if(isinstance(data[i][j], str)):
        continue
      else
        xarr.append(data[i][j])
        yarr.append(i)
    mu_ = mean_imput(xarr, yarr)
    for i in range(r):
      if(isinstance(data[i][j], str)):
        data[i][j] = mu_    # warr is a numpy array  

elapsed = timeit.default_timer() - start_time
print(elapsed)

8.460000026389025e-05


The following cell implements **Zero Imputation**, i.e., replaces empty entries with zeros in the given dataset.

In [16]:
start_time = timeit.default_timer()
for i in range(r):
  for j in range(c):
    if(data[i][j] == '-'):
      data[i][j] = float(0.0)
elapsed = timeit.default_timer() - start_time
print(elapsed)

0.004130086999964533


**Checking for any further missing values**

In [5]:
start_time = timeit.default_timer()
cnt = 0
for i in range(r):
  for j in range(c):
    if(isinstance(data[i][j],str)):
      cnt = cnt+1
if(cnt==0):
  print("No more missing values")
elapsed = timeit.default_timer() - start_time
print("Time taken for checking any missing values: ",elapsed,"sec")

No more missing values
Time taken for checking any missing values:  0.0038982989999567508 sec


The following two cells are for normalizing the entries of the dataset. **For this assignment, we are sticking to normalizing the rows (i.e., the records or samples)**.

Normalizing **only the rows** of the data entries

In [6]:
start_time = timeit.default_timer()
for i in range(r):
  data[i] /= np.linalg.norm(data[i])
elapsed = timeit.default_timer() - start_time
print("Time elapsed to normalize the rows of the entire data: ",elapsed,"sec")

Time elapsed to normalize the rows of the entire data:  0.003617752999957702 sec


Normalizing **only the columns** of the data entries

In [10]:
start_time = timeit.default_timer()
for j in range(c):
  s = 0
  for i in range(r):
    s += data[i][j]**2
  s = math.sqrt(s)
  for i in range(r):
    data[i][j] /= s
elapsed = timeit.default_timer() - start_time
print(elapsed)

0.00529506600000218


### **Step-1** 

Function for **k-Means Clustering** to be applied on the given dataset:

In [7]:
start_time = timeit.default_timer()
def kmeans_cluster(k, randk_, Tmax, data_):
  r_, c_ = data_.shape
  mu = np.zeros((k, c_)) # numpy array containing means of clusters we are building
  for i in range(k):    # initializing the means
    for j in range(c_):
      mu[i][j] = data_[randk_[i]][j]

  cluster = dict()
  basket = [[] for mill in range(k)]
  curr_basket = basket
  for it in range(Tmax):
    for i in range(r_):
      mindist = np.inf   # for maximum similarity, least distance
      muidx = -1
      for kup in range(k):
        sim_ = float(np.dot(data_[i], mu[kup])/(np.linalg.norm(data_[i])*np.linalg.norm(mu[kup])))
        # print("sim = ", sim_)
        if(sim_ >= 1.0):
          sim_ = 1.0
        # dist_ = float(math.sqrt(2*(1.0-sim_)))
        dist_ = float(1.0-sim_)
        if(dist_< mindist):
          mindist = dist_
          muidx = kup
      basket[muidx].append(i)
      cluster[i] = muidx
    
    # computing new means
    mu = np.zeros((k, c_))
    for kup in range(k):
      for id in basket[kup]:
        for j_ in range(c_):
          mu[kup][j_] = mu[kup][j_] + data_[id][j_]
      mu[kup] = mu[kup]/len(basket[kup])
    
    curr_basket = basket
    basket = [[] for mill in range(k)]

  return curr_basket, cluster
elapsed = timeit.default_timer() - start_time
print(elapsed)

0.000545464999959222


### **Step-2**

Function for **Silhouette Score (Coefficient)** computation for the obtained clustering

In [8]:
start_time = timeit.default_timer()
def silhouette_score(k, data_, curr_basket_, cluster_):
  r_, c_ = data_.shape
  sil_sc = np.array([0.0 for i in range(r_)])   # array of silhouette scores for all the r = 143 samples
  cntarr = np.array([0 for i in range(k)])
  for i in range(r_):
    if(len(curr_basket_[cluster_[i]]) == 1):
      sil_sc[i] = 0.0
      continue
    muidx = cluster_[i]
    cntarr[muidx] += 1
    s1 = 0.0
    # print("s1 before = ", type(s1))
    for id in curr_basket_[muidx]:
      if(id!=i):
        # print("type of dot product: ", type(np.dot(data_[id], data_[i])/(np.linalg.norm(data_[i])*np.linalg.norm(data_[id]))))
        sim_ = float(np.dot(data_[id], data_[i])/(np.linalg.norm(data_[i])*np.linalg.norm(data_[id])))
        if(sim_ >= 1.0):
          sim_ = 1.0
        # dist_ = float(math.sqrt(2*(1.0-sim_)))
        dist_ = float(1.0-sim_)
        s1 = s1 + dist_
    # print("s1 after adding = ", type(s1))
    a = s1/(len(curr_basket_[muidx])-1)
    # print("a = ", a)
    
    mins2 = np.inf
    cntkup = 0
    for kup in range(k):
      if(kup != muidx):
        cntkup +=1
        s2 = 0.0
        for id in curr_basket_[kup]:
          sim_ = float(np.dot(data_[id], data_[i])/(np.linalg.norm(data_[i])*np.linalg.norm(data_[id])))
          if(sim_ >= 1.0):
            sim_ = 1.0
          # dist_ = float(math.sqrt(2*(1.0-sim_)))
          dist_ = float(1.0-sim_)
          s2 = s2 + dist_
        s2 = s2/len(curr_basket_[kup])
        if(s2 < mins2):
          mins2 = s2
    b = mins2
    sil_sc[i] = (b-a)/max(a, b)
  
  return sil_sc
elapsed = timeit.default_timer() - start_time
print(elapsed)

0.0008466639999369363


#### The following cell implements **Step 1 and Step 2 together, for $k = 3$**.

In [91]:
start_time = timeit.default_timer()
ts = start_time
k = 3
Tmax = 20
randk = np.unique(np.random.choice(r, k, replace=False))   # array of k random starting seed indices 
print("Starting indices:", randk)
curr_basket, cluster = kmeans_cluster(k, randk, Tmax, data) # kmeans clustering done
elapsed = timeit.default_timer() - start_time
print("Time elapsed for kmeans clustering: ", elapsed,'\n')
# curr_basket.sort()
for kup in range(k):
#     curr_basket[kup].sort()
    print("Length of cluster",str(kup),"= ", len(curr_basket[kup]))
# print(curr_basket)
print('\n')
start_time = timeit.default_timer()
silhouette_arr = silhouette_score(k, data, curr_basket, cluster)
sil_sc = np.mean(silhouette_arr)
elapsed = timeit.default_timer() - start_time
print("Time elapsed for computing Silhouette Coefficient: ", elapsed)
print("Silhouette Coefficient = ",sil_sc)

start_time = timeit.default_timer()
curr_basket.sort()
s1 = ''
for i in range(k):
    curr_basket[i].sort()
    l = len(curr_basket[i])
    for j in range(l-1):
        s1 = s1 + str(curr_basket[i][j])+','
    s1 = s1 + str(curr_basket[i][l-1])+'\n'
# print(s1)
with open("kmeans_k=3.txt", "w") as f:
    f.write(s1)
f.close()
elapsed = timeit.default_timer() - start_time
print("Time elapsed for saving as a text file ('kmeans.txt'): ", elapsed)

cluster_3 = cluster
curr_basket_3 = curr_basket   ## we are saving the current clustering as curr_basket_3 
                              ## so that we can use it in step 4 if k=3 is optimal no of clusters
print("Total time for this cell: ", timeit.default_timer() - ts)                    

Starting indices: [ 50 109 112]
Time elapsed for kmeans clustering:  0.28588430999843695 

Length of cluster 0 =  60
Length of cluster 1 =  69
Length of cluster 2 =  14


Time elapsed for computing Silhouette Coefficient:  0.40547618599885027
Silhouette Coefficient =  0.743572331759283
Time elapsed for saving as a text file ('kmeans.txt'):  0.001441120999515988
Total time for this cell:  0.6964116169983754


### **Step-3 : Finding optimal $k$**

The following cell implements k-Means Clustering and computes the corresponding Silhouette coefficient on the given data.\
For each value of $k$ ($k \in [3, 4, 5, 6]$), we perform 25 experiments; where each experiment generates a random set of $k$ indices and applies *kmeans_cluster()* function on the dataset to cluster them using the Lloyd's algorithm, and then the *silhouette_score()* function computes the Silhouette score and coefficient (**SC**) for this clustering. \
The value of $k$ for which the SC coefficient is highest, is chosen as the optimal $k$ (to be used in Step 4). \
If the optimal $k$ is greater than $3$, we can uncomment the statements in the following code cell after printing *optk*, and thus the corresponding best clustering can be saved for later use. 

In [21]:
## currently we are normalizing each row
karr = [3, 4, 5, 6]
Tmax = 20
nexp = 25
optk = -1
bestsc = -1
for k in karr:
  start_time = timeit.default_timer()
  mean_sil_score = 0.0
  for exp in range(nexp):
    randk = np.unique(np.random.choice(r, k, replace=False))   # array of k random starting seed indices 
    
    curr_basket, cluster = kmeans_cluster(k, randk, Tmax, data)  # performing kmeans
    
    silhouette_arr = silhouette_score(k, data, curr_basket, cluster) # computing the array of silhouette scores
    sil_sc = np.mean(silhouette_arr)
    
    mean_sil_score += float(1.0*sil_sc)
  mean_sil_score /= nexp

  elapsed = timeit.default_timer() - start_time
    
  print("Time elapsed for",str(nexp),"experiments for k =",str(k),": ",elapsed)
  if(bestsc < mean_sil_score):
    bestsc = mean_sil_score
    optk = k
  
  print("Mean Silhouette Coefficient (SC) for k =", str(k),"over", str(nexp), "runs: ", mean_sil_score)

print("Optimal k =",optk)
# print("For this optimal k, we will save the clustering information in a file ('kmeans.txt')\n")

# start_time = timeit.default_timer()
# randk = np.unique(np.random.choice(r, optk, replace=False))   # array of k random starting seed indices 
# # print("Starting indices:", randk)
# curr_basket, cluster = kmeans_cluster(optk, randk, Tmax, data)
# curr_basket.sort()
# s1 = ''
# for i in range(optk):
#     l = len(curr_basket[i])
#     for j in range(l-1):
#         s1 = s1 + str(curr_basket[i][j])+','
#     s1 = s1 + str(curr_basket[i][l-1])+'\n'
# print(s1)
# with open("kmeans.txt", "w") as f:
#     f.write(s1)
# f.close()
# elapsed = timeit.default_timer() - start_time
# print("Time elapsed for clustering, and then saving as a text file ('kmeans.txt'): ", elapsed)

Time elapsed for 25 experiments for k = 3 :  17.92366932799996
Mean Silhouette Coefficient (SC) for k = 3 over 25 runs:  0.6672782988299617
Time elapsed for 25 experiments for k = 4 :  20.10370742499981
Mean Silhouette Coefficient (SC) for k = 4 over 25 runs:  0.6297204526544806
Time elapsed for 25 experiments for k = 5 :  22.169331929000236
Mean Silhouette Coefficient (SC) for k = 5 over 25 runs:  0.6314927049577947
Time elapsed for 25 experiments for k = 6 :  23.741074625999772
Mean Silhouette Coefficient (SC) for k = 6 over 25 runs:  0.6179084279443592
Optimal k = 3


### **Step - 4: Hierarchical Clustering (Single Linkage Agglomerative (Bottom-Up))**

Function to compute **Single Linkage distance** between two clusters

In [26]:
start_time = timeit.default_timer()
def calsingledist(dat, arr1, arr2):     ## computes single linkage distance between two clusters
  mindist2 = np.inf
  for x1 in arr1:
    for x2 in arr2:
      dist2 = float(1.0-float(np.dot(dat[x1], dat[x2])/(np.linalg.norm(dat[x1])*np.linalg.norm(dat[x2]))))
      if (mindist2 > dist2):
        mindist2 = dist2
  return mindist2

# def calcompletedist(dat, arr1, arr2):   ## computes complete linkage distance between two clusters 
#   maxdist2 = -1
#   for x1 in arr1:
#     for x2 in arr2:
#       dist2 = float(math.sqrt(2*1.0-float(np.dot(dat[x1], dat[x2])/(np.linalg.norm(dat[x1])*np.linalg.norm(dat[x2])))))
#       if (maxdist2 < dist2):
#         maxdist2 = dist2
#   return maxdist2
elapsed = timeit.default_timer() - start_time
print(elapsed)

0.0002365909995205584


The following code cell is a function for **Single-Linkage Agglomerative (Bottom-Up) Clustering Algorithm**

In [27]:
start_time = timeit.default_timer()
def bottom_up_single_linkage(k, data_):
  r_, c_ = data_.shape
  curr = r # current number of clusters, initialized to 143
  # print("Initial number of clusters = ", curr)
  clust = [[i] for i in range(r_)]
  # random.shuffle(clust)
  # print("The initial clusters are: ", clust)
  while(curr > k):    # k is the number of clusters at which we should stop
    distmin = np.inf
    c1 = -1
    c2 = -1           # c1 and c2 are indices for which two clusters are getting combined
    for i in range(curr):
      for j in range(i+1, curr):
        # we have singled out two clusters
        # now we find the minimum distance between them
        # i.e., w.r.t, cosine similarity, what is the maximum similarity we can have for two clusters
        # print(i, j)
        dist3 = calsingledist(data_, clust[i], clust[j])
        if(dist3 < distmin):
          distmin = dist3
          c1 = i
          c2 = j
    # print("hello!!")
    # print("Current no of clusters curr)
    clust[c1] = clust[c1] + clust[c2]
    clust.pop(c2)
    curr = curr-1
  return clust
elapsed = timeit.default_timer() - start_time
print(elapsed)


0.00035450300038064597


**Using the *bottom_up_single_linkage()* function for performing the Hierarchical clustering for optimal value of $k$**

In [28]:
start_time = timeit.default_timer()
# k = optk
clust_hier = bottom_up_single_linkage(optk, data)
print("Sample split for k =", str(k),": ", end='')
for i in range(len(clust_hier)):
  print(len(clust_hier[i]),end=' ')
print('\n')
elapsed = timeit.default_timer() - start_time
print(elapsed)

Sample split for k = 3 : 130 6 7 

29.25832234599966


The following cell is for computing the Jaccard Similarity Score between the cluster sets computed using **k-Means Algorithm**, and the cluster sets computed using **Agglomerative (Bottom-Up) Single-Linkage hierarchical clustering algorithm**

In [29]:
start_time = timeit.default_timer()
best_k = optk
## Now given clust and curr-basket
## we have to compute the jaccard similarity between different sets
## clustered by the two algorithms
def jaccard_score(k, curr_basket_, clust_hier_):
  clustmap = dict()
  clust_score = dict()
  indarr = []
  for i in range(k):   # initializing the setwise mapping to -1 for all sets
    clustmap[i] = -1
  for i in range(k):
    basket = curr_basket_[i]
    maxjs = -1
    maxind = -1
    for j in range(k):
      if(j not in indarr):
        cl = clust_hier_[j]
        tmp1 = set(basket)
        tmp2 = set(cl)
        js = len(tmp1.intersection(tmp2))/len(tmp1.union(tmp2))
        if(js > maxjs):
          maxjs = js
          maxind = j
    indarr.append(maxind)
    clustmap[i] = maxind
    clust_score[i] = maxjs
  return clustmap, clust_score
elapsed = timeit.default_timer() - start_time
print(elapsed)

0.0013405820000116364


The following is **Jaccard Similarity score comparison** for k-Means and Bottom Up Single Linkage clustering. 

In [82]:
start_time = timeit.default_timer()
## Now given cluster_3 and curr_basket_3
## we have to compute the jaccard similarity between different sets
## clustered by the two algorithms
best_k = optk
clust_hier.sort()
clustermap, jmap = jaccard_score(best_k, curr_basket_3, clust_hier)  ## computation of Jaccard Score
print("Cluster Map: ", clustermap)
print("Jaccard Score recorded: ", jmap)
elapsed = timeit.default_timer() - start_time
print(elapsed)

Cluster Map:  {0: 0, 1: 1, 2: 2}
Jaccard Score recorded:  {0: 0.4580152671755725, 1: 0.0, 2: 0.5}
0.017025641999680374


Saving the cluster information from **Agglomerative Clustering** to ***agglomerative.txt***

In [84]:
ts = timeit.default_timer()
s1 = ''
clust_hier.sort()
for i in range(k):
    clust_hier[i].sort()
    l = len(clust_hier[i])
    for j in range(l-1):
        s1 = s1 + str(clust_hier[i][j])+','
    s1 = s1 + str(clust_hier[i][l-1])+'\n'
# print(s1)
with open("agglomerative.txt", "w") as f:
    f.write(s1)
f.close()
print("Time taken to save hierarchical clustering to text file: ", timeit.default_timer() - ts,"sec")

Time taken to save hierarchical clustering to text file:  0.0031453900000997237 sec


The following cell was used to test the implementation of the Bottom-Up Single Linkage algorithm using *sklearn*

In [33]:
# from sklearn.cluster import AgglomerativeClustering
# # import numpy as np
# X = data
# n_clust = optk
# clustering = AgglomerativeClustering(n_clusters = n_clust, linkage = "single", metric="cosine").fit(X)
# print(clustering)
# # AgglomerativeClustering()
# print(type(clustering.labels_))
# # array([1, 1, 1, 0, 0, 0])

# cnt = [0 for kup in range(n_clust)]
# n = len(clustering.labels_)
# for i in range(n):
#   cnt[clustering.labels_[i]] += 1
# print(cnt)


AgglomerativeClustering(linkage='single', metric='cosine', n_clusters=3)
<class 'numpy.ndarray'>
[130, 7, 6]
